In [ ]:
import json
import os

import altair as alt
import pandas as pd

In [ ]:
# alt.data_transformers.enable('csv')
alt.data_transformers.disable_max_rows()

In [ ]:
DATA_DIR = '../outputs/inview'

In [ ]:
failed_videos = {}
for f in os.listdir(DATA_DIR):
    if not f.startswith('test-inview--failed_videos'): continue

    with open(os.path.join(DATA_DIR, f), 'r') as _f:
        content = json.load(_f)

    name = f[len('test-inview--failed_videos--'):-len('.json')]
    failed_videos[name] = []
    for v_name, err in content:
        failed_videos[name].append(v_name)

In [ ]:
stage_order = [
    'InView', 'InViewOld', 'DecodeFrame', 'Detection2D.YoloDetection', 'Detection2D.ObjectTypeFilter',
    'DepthEstimation', 'Detection3D.FromDetection2DAndDepth', 'Detection3D.FromDetection2DAndRoad',
    'DetectionEstimation', 'Tracking2D.StrongSORT', 'Tracking3D.FromTracking2DAndDepth',
    'Tracking3D.FromTracking2DAndRoad', 'SegmentTrajectory.FromTracking3D'
]

test_names = {
    'de': 'Only Detection Estimation',
    'noopt': 'Baseline',
    'inview': 'Only In-View',
    'geo': 'Only Geo Depth Estimation',
    'objectfilter': 'Only Object Filter',
    'opt': 'Optimized',
    'optde': 'Optimized with Detection Estimation',
    'inview-new': 'New In-View',
    'inview-old': 'Old In-View',
    'inview-old2': 'Old In-View - minDistance',
}

test_order = [
    'noopt',
    'inview',
    'objectfilter',
    'geo',
    'de',
    'opt',
    'optde'
]

In [ ]:
def combine_perf():
    data = []
    for filename in os.listdir(DATA_DIR):
        if filename.startswith('test-inview--perf'):
            print(filename)
            test = filename.split("--")[2].split(".")[0]
            with open(os.path.join(DATA_DIR, filename), 'r') as f:
                for stage in json.load(f):
                    stagename = stage['stage']
                    for b in stage['benchmark']:
                        after, before = b['keep']
                        assert after <= before
                        skip = (1. - (after / before)) if before != 0 else 0
                        data.append({
                            **b,
                            'stage': (stagename
                                      .replace('Detection2D', 'D2D')
                                      .replace('Detection3D', 'D3D')
                                      .replace('Tracking2D', 'T2D')
                                      .replace('Tracking3D', 'T3D')
                                      .replace('SegmentTrajectory', 'ST')),
                            '_test': test,
                            'test': test_names[test],
                            'test_order': 0, # test_order.index(test),
                            'name': b['name'].split('/')[-1].split('.')[0],
                            'skip': skip,
                            'stage_order': stage_order.index(stagename)
                        })
    return data


perf = combine_perf()

In [ ]:
# with open('./output/perf.json', 'w') as f:
#     json.dump([p for p in perf if p['run'] == "0"], f, indent=1)

In [ ]:
perf[0]

In [ ]:
for name, videos in failed_videos.items():
    if len(videos) != 0:
        print(name, videos)

In [ ]:
df_perf = pd.DataFrame.from_dict(perf)
# df_perf = df_perf[df_perf['run'] == "0"]
df_perf[:10]

In [ ]:
df_perf.groupby(['test', 'stage'])[['name']].count()

In [ ]:
df_new = df_perf[df_perf._test == 'inview-new']
df_old = df_perf[df_perf._test == 'inview-old']
df_old2 = df_perf[df_perf._test == 'inview-old2']

In [ ]:
indices = ['name']

df = (df_new
    .set_index(indices)
    .join(df_old.set_index(indices), lsuffix='_new', how='inner')
    .join(df_old2.set_index(indices), lsuffix='_old', rsuffix='_old2', how='inner')
    .reset_index()
    [['name', 'runtime_new', 'skip_new', 'runtime_old', 'skip_old', 'runtime_old2', 'skip_old2']]
)
df

In [ ]:
(alt.Chart(df_perf, title="Average Runtime for each test", height=300, width=800)
    .mark_bar()
    .encode(x='average(runtime)', y='test:O')
)

In [ ]:
brush = alt.selection(type='interval', resolve='global')
base = (alt.Chart(df)
    .mark_point(opacity=0.1)
    .encode(
        alt.Y('runtime_new:Q'),
        color=alt.condition(brush, alt.ColorValue('steelblue'), alt.ColorValue('gray')),
    )
    .add_selection(brush)
)
scale = alt.Scale(domain=[0, 0.6])
base.encode(alt.X('runtime_old:Q', scale=scale)) | base.encode(alt.X('runtime_old2:Q', scale=scale, title="runtime_old - minDistance"))

In [ ]:
base = (alt.Chart(df_perf)
    .encode(
        alt.Y('test:O'),
        alt.X('runtime:Q')
    )
)

base.mark_boxplot() & base.mark_tick(opacity=0.02)

In [ ]:
base = (alt.Chart(df_perf)
    .encode(
        alt.Y('test:O'),
        alt.X('skip:Q', scale=alt.Scale(domain=[0, 1]), title="Skip Ratio")
    )
)

base.mark_boxplot() & base.mark_tick(opacity=0.2) & base.mark_errorbar(extent='ci')

In [ ]:
alt.Chart(df).mark_point().encode(
    alt.X(alt.repeat("column"), type='quantitative'),
    alt.Y(alt.repeat("row"), type='quantitative'),
).properties(
    width=150,
    height=150
).repeat(
    row=['runtime_new', 'runtime_old', 'runtime_old2'],
    column=['runtime_old2', 'runtime_old', 'runtime_new']
).interactive()

In [ ]:
(
    alt.Chart(
        df_perf,
        title='CDF of skip ratio'
    )
    .transform_window(
        ECDF="cume_dist()",
        groupby=["test"],
        sort=[{"field": "skip"}],
    )
    .mark_line()
    .encode(
        alt.X("skip:Q", title='Frame Skip Ratio'),
        alt.Y("ECDF:Q"),
        color="test:N"
    )
)

In [ ]:
(
    alt.Chart(
        df_perf,
        title='CDF of Runtime'
    )
    .transform_window(
        ECDF="cume_dist()",
        groupby=['test'],
        sort=[{"field": "runtime"}],
    )
    .mark_line()
    .encode(
        x=alt.X("runtime:Q", title='Runtime (ms)'),
        y="ECDF:Q",
        color='test:N'
    )
)


In [ ]:
base = (
    alt.Chart(
        df_perf,
        title='CDF of Runtime',
        width=350
    )
    .transform_window(
        ECDF="cume_dist()",
        groupby=['test'],
        sort=[{"field": "runtime"}],
    )
    .mark_line()
    .encode(
        x=alt.X("runtime:Q", title='Runtime (ms)'),
        y="ECDF:Q"
    )
)

(
    base.transform_filter("datum.test === 'New In-View'").properties(title='New In-View') |
    base.transform_filter("datum.test === 'Old In-View'").properties(title='Old In-View') |
    base.transform_filter("datum.test === 'Old In-View - minDistance'").properties(title='Old In-View - minDistance')
)